# BiblioTECHas

In [38]:
from pymatgen.core import Lattice, Structure 
import numpy as np 
from pymatgen.core.composition import Element, Composition
from pymatgen.core.periodic_table import Specie
import math
import random
from pymatgen.io.vasp.inputs import Poscar, Kpoints, Potcar,Incar
from pymatgen.io.vasp.outputs import Outcar 
from pymatgen.entries.computed_entries import ComputedStructureEntry
import os
import clease
from clease.settings import Concentration
import numpy as np
from clease.settings import CEBulk
from clease.structgen import NewStructures
from ase.db import connect
from ase.io import write
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.transformations.standard_transformations import PerturbStructureTransformation
from pymatgen.io.cif import CifWriter
import json
# now lets make the computed entries from chgnet potential 
from chgnet.model.model import CHGNet 
from chgnet.model.dynamics import CHGNetCalculator 
from chgnet.model import StructOptimizer

# Functions

In [41]:
def create_cca_primitive(comp_list, a, prim = True):
    """
    Create a CCA (Complex Concentrated Alloy) primitive structure using the given atom dictionary and lattice constant.

    Parameters:
        comp_list (dict): A dictionary containing the atomic fraction of the material you want.
        a (float): The lattice constant of the cubic cell.
        prim (bool): Whether to create a primitive cell or not. Default is True and will create a primitive cell with one atom.

    Returns:
        pymatgen.core.structure.Structure: The CCA supercell structure.

    """
    direct_coords = [[0,0,0]]
    if prim:
        test_bcc = Structure(Lattice.cubic(a), [comp_list], direct_coords)
    else:
        test_bcc = Structure.from_spacegroup("Im-3m",Lattice.cubic(a),[comp_list],direct_coords)
    return test_bcc

def _closest_composition(comp, num_atoms, bal_element):
    """
    Calculate the closest composition of elements given a target number of atoms,
    adjusting the balance element to handle any shortfall or excess.
    
    Parameters:
    comp (dict): A dictionary representing the composition of elements, where the keys are the element symbols and the values are the fractions.
    num_atoms (int): The target number of atoms.
    bal_element (str): The symbol of the element used to adjust the balance so that the total number of atoms is correct.

    Returns:
    tuple: A tuple containing two dictionaries. The first dictionary represents the number of atoms for each element, with adjustments made through the balance element. The second dictionary represents the actual fractions of each element, rounded to 5 decimal places.
    """
    # Normalize the composition fractions
    total_fraction = sum(comp.values())
    normalized_comp = {element: fraction / total_fraction for element, fraction in comp.items()}
    
    # Calculate initial atoms for each element, excluding the balance element, rounding to nearest integer.
    atoms = {}
    for element, fraction in normalized_comp.items():
        if element != bal_element:
            atoms[element] = round(fraction * num_atoms)
    
    # Calculate the number of atoms assigned so far and adjust the balance element accordingly.
    assigned_atoms = sum(atoms.values())
    atoms[bal_element] = num_atoms - assigned_atoms
    
    # Recalculate the actual fractions to ensure they sum to 1
    actual_fractions = {element: round(atoms[element] / num_atoms, 3) for element in atoms}
    
    return atoms, actual_fractions

def closest_composition(comp, num_atoms, bal_element):

    # first sum up the fractions of the composition that are not the balance element
    total_fraction = sum([comp[element] for element in comp if element != bal_element])

    # now set the balance element to the difference between the total fraction and 1
    comp[bal_element] = 1 - total_fraction

    # now we need to normalize the composition to the number of atoms, ignoring the balance element
    # we will round up to the nearest integer for the non balance elements

    # first calculate the number of atoms for each element
    atoms = {element: math.ceil(comp[element] * num_atoms) for element in comp if element != bal_element}

    # then calculate the number of atoms assigned so far

    assigned_atoms = sum(atoms.values())

    # now we need to adjust the balance element to make sure the total number of atoms is correct
    atoms[bal_element] = num_atoms - assigned_atoms

    # now we need to recalculate the actual fractions to make sure they sum to 1
    # let's first start with calculating the fractions of the non balance elements and rounding to 3 decimals
    actual_fractions = {element: round(atoms[element] / num_atoms, 3) for element in atoms if element != bal_element}

    # now we need to calculate the fraction of the balance element by subtracting the sum of the fractions of the non balance elements from 1
    actual_fractions[bal_element] = round(1 - sum(actual_fractions.values()), 3)

    return atoms, actual_fractions




def generate_compositions(atom_dict, n, balance_element, seed=42):
    """
    Generate random compositions based on given atom_dict and balance_element.

    Parameters:
    - atom_dict (dict): A dictionary containing the elements as keys and their corresponding minimum and maximum values as values.
    - n (int): The number of compositions to generate.
    - balance_element (str): The element that needs to be balanced in the compositions.
    - seed (int): The seed value for the random number generator. Default is 42.

    Returns:
    - samples (list): A list of dictionaries representing the generated compositions.
    """

    random.seed(seed)
    samples = []

    for _ in range(n):
        sample = {}
        total_percentage = 0.0

        # Randomly sample for each element except the balance element
        for element, (min_val, max_val) in atom_dict.items():
            if element != balance_element:
                sample[element] = round(random.uniform(min_val, max_val), 4)
                total_percentage += sample[element]

        # Adjust if sum of other elements exceeds 1
        if total_percentage >= 1.0:
            # Scale down the sampled values proportionally if the total exceeds 100%
            scale_factor = (1.0 - atom_dict[balance_element][0]) / total_percentage
            for element in sample:
                sample[element] = round(sample[element] * scale_factor, 4)
            total_percentage = sum(sample.values())

        # Ensure the balance element's value makes the total sum 1
        sample[balance_element] = round(1.0 - total_percentage, 4)

        # Check if balance is less than its min value, adjust if necessary
        if sample[balance_element] < atom_dict[balance_element][0]:
            # This scenario should be rare with proper input ranges
            # An adjustment strategy might be needed depending on specific requirements
            raise ValueError(f"Cannot meet balance element '{balance_element}' minimum requirement without exceeding total of 1.0")

        samples.append(sample)
        
    return samples


def create_random_supercells(composition,alat,supercell_size,db_name,num_structures):
    # A_eq is the identity matrix with a size equal to the number of elements in the composition, i.e len(composition.keys())
    A_eq = np.eye(len(composition.keys()))
    b_eq = list(composition.values())
    conc = Concentration(basis_elements=[list(composition.keys())],
                         A_eq = A_eq,
                         b_eq = b_eq)
    
    settings = CEBulk(crystalstructure='bcc',
                      a=alat,
                      size=[supercell_size,supercell_size,supercell_size],
                      concentration=conc,
                      db_name=db_name,
                      max_cluster_dia=[6.0, 4.5, 4.5])
    


    ns = NewStructures(settings, generation_number=1,
                    struct_per_gen=num_structures)
    ns.generate_random_structures()

def connect_ase_db(db_name):
    from ase.db import connect
    atoms = connect(db_name)
    print(len(atoms))
    for idx in range(len(atoms)):
        curr_struct = atoms.get(id=idx+1).toatoms()
        print(curr_struct)
    return atoms

        
def ase_db_to_pymatgen(db_name,output_file=None):

    data = {}
    Adaptor = AseAtomsAdaptor()
    atoms = connect(db_name) 

    for idx in range(2,len(atoms)+1):
        
        curr_struct = atoms.get(id=idx).toatoms()
        
        curr_pymatgen = Adaptor.get_structure(curr_struct)
        
        
        trans = PerturbStructureTransformation(distance=0.1, min_distance=0.01)
        distorted_struct = trans.apply_transformation(curr_pymatgen)
        
        data.update({f'{idx}':distorted_struct.as_dict()})
        
    if output_file is not None:
        with open(output_file, 'w') as f:
            json.dump(data, f,)
    else:
        return data
        
def generate_random_supercells(composition, num_structures, lattice_parameter=3.01, supercell_size=4,supercell_type='cubic', seed=42):
    random.seed(seed) 
    supercells = []

    comp_list = [key for key in composition]
    for _ in range(num_structures):
        # Create a bulk V crystal with the specified lattice parameter
        #prim_cell = bulk(comp_list[0], cubic=True, a=lattice_parameter)
        if supercell_type == 'cubic':
            prim_cell = Structure(Lattice.cubic(lattice_parameter), [comp_list[0],comp_list[0]], [[0, 0, 0],[0.5, 0.5, 0.5]])
        elif supercell_type == 'prim':
            prim_cell = Structure(Lattice.cubic(lattice_parameter), [comp_list[0]], [[0, 0, 0]])
        
        # make the supercell
        supercell = prim_cell * (supercell_size,supercell_size,supercell_size)

        # Create a list of all possible indices for the current element
        all_indices = list(range(len(supercell.sites)))

        for element, count in composition.items():
            # Randomly select 'count' indices for the current element
            selected_indices = random.sample(all_indices, count)
            
            # Change the selected indices to the current element
            for index in selected_indices:
                supercell.replace(index,element)
            
            # remove the selected_indices from the all_indices list
            all_indices = [index for index in all_indices if index not in selected_indices]
        
        supercell = supercell.get_sorted_structure()

        supercells.append(supercell)

    return supercells

def create_computed_entry_from_outcar(outcar,poscar=None):
    """
    Create a ComputedEntry from the given OUTCAR file."""
    # need to test this with an outcar 
    outcar = Outcar(outcar)
    energy = outcar.final_energy
    composition = outcar.structure.composition
    if Poscar is not None:
        poscar = Poscar.from_file(poscar)
        structure = poscar.structure
    else:
        structure = outcar.structure

    return ComputedStructureEntry(composition, energy, structure = structure)

def create_computed_entry_from_chgnet(structure, energy):
    composition = structure.composition
    return ComputedStructureEntry(composition=composition, energy = energy, structure=structure)
    


# Creating Primitive and SuperCells

## Create the Primitive Structure

In [68]:
# create a primitive structure
a = 3.01 
x_cr = 0.02
x_ti = 0.02
x_zr = 0.01
x_w = 0.04
x_v = round(1 - x_cr - x_ti - x_w - x_zr,5)
#x_v = round(1 - x_cr - x_ti - x_zr, 5)

composition = {'V': x_v, 'Cr': x_cr, 'Ti': x_ti, 'W': x_w, 'Zr': x_zr}
#composition = {'V': x_v, 'Cr': x_cr, 'Ti': x_ti, 'Zr': x_zr}
num_atoms = 64
atom_dict, actual_comp = closest_composition(composition, num_atoms=num_atoms, bal_element='V')

In [46]:
from pymatgen.core import Lattice, Structure
from pymatgen.core.periodic_table import Specie
from pymatgen.core.composition import Composition

def create_cca_primitive(comp_list, a, prim=True):
    # Create the species composition for each site as a Composition object
    # We need to pass the composition as a list of dicts where each dict represents the species and their fraction at that site
    species_composition = Composition(comp_list)

    # Coordinates for the primitive cell, assuming a BCC structure
    coords = [[0, 0, 0], [0.5, 0.5, 0.5]]  # Two distinct positions in BCC

    if prim:
        # Define the primitive lattice vectors for BCC
        lattice_vectors = [
            [0.5 * a, 0.5 * a, 0.5 * -a],
            [0.5 * a, 0.5 * -a, 0.5 * a],
            [0.5 * -a, 0.5 * a, 0.5 * a]
        ]

        # Create the lattice
        lattice = Lattice(lattice_vectors)
        # Initialize the structure with the same composition for each of the two sites
        structure = Structure(lattice, [species_composition], [[0,0,0]])
    else:
        # For non-primitive, we use the same approach but might specify a different spacegroup or adjustments
        lattice = Lattice.cubic(a)
        structure = Structure.from_spacegroup("Im-3m", lattice, [species_composition], [[0,0,0]])

    return structure

# Example usage
composition = {'V': 0.580, 'Cr': 0.1, 'Ti': 0.05, 'W': 0.1, 'Zr': 0.1}
a = 3.01

updated_atoms, updated_comp = closest_composition(composition, num_atoms=64, bal_element='V')

print(updated_atoms)

print(updated_comp)


{'Cr': 7, 'Ti': 4, 'W': 7, 'Zr': 7, 'V': 39}
{'Cr': 0.109, 'Ti': 0.062, 'W': 0.109, 'Zr': 0.109, 'V': 0.611}


In [48]:
test = create_cca_primitive(updated_comp, a)
print(test)
writer = CifWriter(test)
file_path = '/Users/myless/Packages/structure_maker/Entries'
# make the filename after the composition from updated_atoms i.e the number of atoms for each element
# for example if {'Cr': 7, 'Ti': 4, 'W': 7, 'Zr': 7, 'V': 39}, it would be named Cr7_Ti4_W7_Zr7_V39.cif
file_name = '_'.join([f'{key}{value}' for key, value in updated_atoms.items()]) + '.cif'
writer.write_file(os.path.join(file_path,file_name))

Full Formula (Zr0.109 Ti0.062 V0.611 Cr0.109 W0.109)
Reduced Formula: Zr0.109Ti0.062V0.611Cr0.109W0.109
abc   :   2.606736   2.606736   2.606736
angles: 109.471221 109.471221 109.471221
pbc   :       True       True       True
Sites (1)
  #  SP                                                a    b    c
---  ----------------------------------------------  ---  ---  ---
  0  Zr:0.109, Ti:0.062, V:0.611, Cr:0.109, W:0.109    0    0    0


In [17]:
from pymatgen.core import Lattice, Structure
import os
# Lattice parameter for BCC iron, as an example
a = 2.87  # in angstroms

# Define the primitive lattice vectors for BCC
lattice_vectors = [
    [0.5 * a, 0.5 * a, 0.5 * -a],
    [0.5 * a, 0.5 * -a, 0.5 * a],
    [0.5 * -a, 0.5 * a, 0.5 * a]
]

# Create the lattice
lattice = Lattice(lattice_vectors)

# Define the structure with one atom at the origin
structure = Structure(lattice, ["Fe"], [[0, 0, 0]])

# Print the structure
print(structure*(2,2,2))

from pymatgen.core import Structure
from pymatgen.io.cif import CifWriter
#read the entry

supercell = structure * (3,3,3)
# write the cif file
writer = CifWriter(supercell)
file_path = '/Users/myless/Packages/structure_maker/Entries'
file_name = 'new_test_fe.cif'
writer.write_file(os.path.join(file_path,file_name))

Full Formula (Fe8)
Reduced Formula: Fe
abc   :   4.970986   4.970986   4.970986
angles: 109.471221 109.471221 109.471221
pbc   :       True       True       True
Sites (8)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Fe    0    0    0
  1  Fe    1    1    0.5
  2  Fe    0    0.5  0
  3  Fe    0    0.5  0.5
  4  Fe    0.5  0    0
  5  Fe    0.5  0    0.5
  6  Fe    0.5  0.5  0
  7  Fe    0.5  0.5  0.5


In [7]:
from pymatgen.core import Lattice, Structure

# Define the lattice parameter, 'a'
a = 2.87  # Example for iron in angstroms

# Create a cubic lattice with the given lattice parameter
lattice = Lattice.cubic(a)

# Define the structure with iron atoms
# We place one iron atom at the origin (corner of the cube)
# and another at the body center of the cube (0.5, 0.5, 0.5 in fractional coordinates)
structure = Structure(lattice, ["Fe", "Fe"], [[0, 0, 0], [0.5, 0.5, 0.5]])*(2,2,2)

# Print the structure
print(structure)
writer = CifWriter(structure)
file_path = '/Users/myless/Packages/structure_maker/Entries'
file_name = 'test_fe_bcc.cif'
writer.write_file(os.path.join(file_path,file_name))

Full Formula (Fe16)
Reduced Formula: Fe
abc   :   5.740000   5.740000   5.740000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (16)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Fe    0     0     0
  1  Fe    0     0     0.5
  2  Fe    0     0.5   0
  3  Fe    0     0.5   0.5
  4  Fe    0.5   0     0
  5  Fe    0.5   0     0.5
  6  Fe    0.5   0.5   0
  7  Fe    0.5   0.5   0.5
  8  Fe    0.25  0.25  0.25
  9  Fe    0.25  0.25  0.75
 10  Fe    0.25  0.75  0.25
 11  Fe    0.25  0.75  0.75
 12  Fe    0.75  0.25  0.25
 13  Fe    0.75  0.25  0.75
 14  Fe    0.75  0.75  0.25
 15  Fe    0.75  0.75  0.75


In [16]:
from ase.build import bulk
from ase.build import make_supercell
from pymatgen.io.ase import AseAtomsAdaptor

# Create a bulk V crystal with the specified lattice parameter
prim_cell = bulk('Fe', crystalstructure='bcc', a=2.87, orthorhombic=True)
print(prim_cell)

# Create a supercell with a 2x2x2 expansion
supercell = make_supercell(prim_cell, [[2, 0, 0], [0, 2, 0], [0, 0, 2]])
structure = AseAtomsAdaptor().get_structure(supercell)
print(structure)
writer = CifWriter(structure)
file_path = '/Users/myless/Packages/structure_maker/Entries'
file_name = 'ase_to_pymat_fe_bcc.cif'
writer.write_file(os.path.join(file_path,file_name))



Atoms(symbols='Fe2', pbc=True, cell=[2.87, 2.87, 2.87], initial_magmoms=...)
Full Formula (Fe16)
Reduced Formula: Fe
abc   :   5.740000   5.740000   5.740000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (16)
  #  SP       a     b     c    magmom
---  ----  ----  ----  ----  --------
  0  Fe    0     0     0          2.3
  1  Fe    0.25  0.25  0.25       2.3
  2  Fe    0     0     0.5        2.3
  3  Fe    0.25  0.25  0.75       2.3
  4  Fe    0     0.5   0          2.3
  5  Fe    0.25  0.75  0.25       2.3
  6  Fe    0     0.5   0.5        2.3
  7  Fe    0.25  0.75  0.75       2.3
  8  Fe    0.5   0     0          2.3
  9  Fe    0.75  0.25  0.25       2.3
 10  Fe    0.5   0     0.5        2.3
 11  Fe    0.75  0.25  0.75       2.3
 12  Fe    0.5   0.5   0          2.3
 13  Fe    0.75  0.75  0.25       2.3
 14  Fe    0.5   0.5   0.5        2.3
 15  Fe    0.75  0.75  0.75       2.3


In [20]:
from pymatgen.core import Lattice, Structure

a = 2.87  # Iron BCC lattice parameter in angstroms

# Primitive lattice vectors
primitive_vectors = [
    [0.5 * a, 0.5 * a, -0.5 * a],
    [0.5 * a, -0.5 * a, 0.5 * a],
    [-0.5 * a, 0.5 * a, 0.5 * a]
]
primitive_lattice = Lattice(primitive_vectors)
primitive_structure = Structure(primitive_lattice, ["Fe"], [[0, 0, 0]])
primitive_supercell = primitive_structure.make_supercell([2, 2, 2])

# Conventional cubic lattice
conventional_lattice = Lattice.cubic(a)
conventional_structure = Structure(conventional_lattice, ["Fe","Fe"], [[0, 0, 0], [0.5, 0.5, 0.5]])
orthorhombic_supercell = conventional_structure.make_supercell([2, 2, 2])

from pymatgen.io.cif import CifWriter

# Export primitive supercell
writer = CifWriter(primitive_supercell)
writer.write_file("primitive_bcc_fe.cif")

# Export orthorhombic supercell
writer = CifWriter(orthorhombic_supercell)
writer.write_file("orthorhombic_bcc_fe.cif")




In [23]:
import json 

single_atom = Structure.from_dict(json.load(open('/Users/myless/Packages/structure_maker/Entries/v1_6cr1_6ti1_6w1_6zr_prim_struct.json','r')))
print(single_atom)



Full Formula (Zr0.01562 Ti0.01562 V0.9375 Cr0.01562 W0.01562)
Reduced Formula: Zr0.01562Ti0.01562V0.9375Cr0.01562W0.01562
abc   :   3.010000   3.010000   3.010000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (1)
  #  SP                                                a    b    c
---  ----------------------------------------------  ---  ---  ---
  0  Zr:0.016, Ti:0.016, V:0.938, Cr:0.016, W:0.016    0    0    0


In [30]:
composition = {'Zr':0.016, 'Ti':0.016, 'V':0.936, 'Cr':0.016, 'W':0.016}
# Define the primitive lattice vectors for BCC
a = 3.01
lattice_vectors = [
    [0.5 * a, 0.5 * a, 0.5 * -a],
    [0.5 * a, 0.5 * -a, 0.5 * a],
    [0.5 * -a, 0.5 * a, 0.5 * a]
]
test_structure = Structure(lattice_vectors, [composition], [[0,0,0]])
print(test_structure)

#supercell_test = test_structure * (4,4,4)
supercell_test = test_structure 

# write the supercell_test as a json file
with open('/Users/myless/Packages/structure_maker/Entries/vcrtiwzr_prim_test.json','w') as f:
    json.dump(supercell_test.as_dict(), f)

Full Formula (Zr0.016 Ti0.016 V0.936 Cr0.016 W0.016)
Reduced Formula: Zr0.016Ti0.016V0.936Cr0.016W0.016
abc   :   2.606736   2.606736   2.606736
angles: 109.471221 109.471221 109.471221
pbc   :       True       True       True
Sites (1)
  #  SP                                                a    b    c
---  ----------------------------------------------  ---  ---  ---
  0  Zr:0.016, Ti:0.016, V:0.936, Cr:0.016, W:0.016    0    0    0


In [70]:
# Example usage
composition = {'V': 0.580, 'Cr': 0.01, 'Ti': 0.01, 'W': 0.03, 'Zr': 0.01}
a = 3.01

print(actual_comp)
primitive_structure = create_cca_primitive(actual_comp, a, prim=False)

print(primitive_structure)


{'Cr': 0.016, 'Ti': 0.016, 'W': 0.047, 'Zr': 0.016, 'V': 0.906}


ValueError: Species occupancies sum to more than 1!

In [55]:
#prim_struct = create_cca_primitive(actual_comp, a) # one atom in the structure
dos_prim_struct = create_cca_primitive(atom_dict, a, prim=False) # this means a two atom primitive structure 

print(dos_prim_struct)

Full Formula (Zr2 Ti2 V116 Cr2 W6)
Reduced Formula: ZrTiV58CrW3
abc   :   3.010000   3.010000   3.010000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (128)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Cr    0    0    0
  1  Cr    0.5  0.5  0.5
  2  Ti    0    0    0
  3  Ti    0.5  0.5  0.5
  4  W     0    0    0
  5  W     0.5  0.5  0.5
  6  W     0    0    0
  7  W     0.5  0.5  0.5
  8  W     0    0    0
  9  W     0.5  0.5  0.5
 10  Zr    0    0    0
 11  Zr    0.5  0.5  0.5
 12  V     0    0    0
 13  V     0.5  0.5  0.5
 14  V     0    0    0
 15  V     0.5  0.5  0.5
 16  V     0    0    0
 17  V     0.5  0.5  0.5
 18  V     0    0    0
 19  V     0.5  0.5  0.5
 20  V     0    0    0
 21  V     0.5  0.5  0.5
 22  V     0    0    0
 23  V     0.5  0.5  0.5
 24  V     0    0    0
 25  V     0.5  0.5  0.5
 26  V     0    0    0
 27  V     0.5  0.5  0.5
 28  V     0    0    0
 29  V     0.5  0.5  0.5
 30  V     0    0    0
 31  V  

In [48]:
for site in dos_prim_struct:
    fractional_comps = site.species
    print(fractional_comps.get_wt_fraction)
    #if sum(fractional_comps.values()) != 1:
        #print('sum of fractions is not equal to 1')

<bound method Composition.get_wt_fraction of Composition('Cr0.01562 Ti0.01562 W0.04688 Zr0.01562 V0.90625')>
<bound method Composition.get_wt_fraction of Composition('Cr0.01562 Ti0.01562 W0.04688 Zr0.01562 V0.90625')>


In [13]:
from pymatgen.core import Structure
from pymatgen.io.cif import CifWriter
#read the entry

prim_entry = json.load(open('/Users/myless/Packages/structure_maker/Entries/vcrtiwzr_prim_entry.json','r'))
print(prim_entry)
prim_structure = Structure.from_dict(prim_entry)

# write the cif file
writer = CifWriter(prim_structure)
file_path = '/Users/myless/Packages/structure_maker/Entries'
file_name = 'v1_6cr1_6ti1_6w1_6zr.cif'
writer.write_file(os.path.join(file_path,file_name))

{'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0.0, 'lattice': {'matrix': [[3.01, 0.0, 0.0], [0.0, 3.01, 0.0], [0.0, 0.0, 3.01]], 'pbc': [True, True, True], 'a': 3.01, 'b': 3.01, 'c': 3.01, 'alpha': 90.0, 'beta': 90.0, 'gamma': 90.0, 'volume': 27.270900999999995}, 'properties': {}, 'sites': [{'species': [{'element': 'V', 'occu': 0.09000000000000008}, {'element': 'Cr', 'occu': 0.02}, {'element': 'Ti', 'occu': 0.02}, {'element': 'Zr', 'occu': 0.01}, {'element': 'W', 'occu': 0.04}], 'abc': [0.0, 0.0, 0.0], 'xyz': [0.0, 0.0, 0.0], 'properties': {}, 'label': 'Zr:0.010, Ti:0.020, V:0.090, Cr:0.020, W:0.040'}, {'species': [{'element': 'V', 'occu': 0.09000000000000008}, {'element': 'Cr', 'occu': 0.02}, {'element': 'Ti', 'occu': 0.02}, {'element': 'Zr', 'occu': 0.01}, {'element': 'W', 'occu': 0.04}], 'abc': [0.5, 0.5, 0.5], 'xyz': [1.505, 1.505, 1.505], 'properties': {}, 'label': 'Zr:0.010, Ti:0.020, V:0.090, Cr:0.020, W:0.040'}]}


In [14]:
one_atom_writer = CifWriter(prim_struct)
file_name = 'v1_6cr1_6ti1_6w1_6zr_one_atom.cif'
one_atom_writer.write_file(os.path.join(file_path,file_name))

In [25]:
two_atom_writer = CifWriter(dos_prim_struct)
file_name = 'v1_6cr1_6ti1_6w1_6zr_dos_atom.cif'
two_atom_writer.write_file(os.path.join(file_path,file_name))

In [15]:
vcrtiwzr_test = Structure.from_file('/Users/myless/Packages/structure_maker/Entries/v1_6cr1_6ti1_6w1_6zr_one_atom.cif')
print(vcrtiwzr_test)

Full Formula (Zr0.20312 Ti0.20312 V0.1875 Cr0.20312 W0.20312)
Reduced Formula: Zr0.20312Ti0.20312V0.1875Cr0.20312W0.20312
abc   :   3.010000   3.010000   3.010000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (1)
  #  SP                                                a    b    c
---  ----------------------------------------------  ---  ---  ---
  0  Zr:0.203, Ti:0.203, V:0.188, Cr:0.203, W:0.203    0    0    0


In [18]:
supercell = vcrtiwzr_test * (4,4,4)
print(supercell)

Full Formula (Zr12.99968 Ti12.99968 V12 Cr12.99968 W12.99968)
Reduced Formula: Zr12.99968Ti12.99968V12Cr12.99968W12.99968
abc   :  12.040000  12.040000  12.040000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (64)
  #  SP                                                 a     b     c
---  ----------------------------------------------  ----  ----  ----
  0  Zr:0.203, Ti:0.203, V:0.188, Cr:0.203, W:0.203  0     0     0
  1  Zr:0.203, Ti:0.203, V:0.188, Cr:0.203, W:0.203  0     0     0.25
  2  Zr:0.203, Ti:0.203, V:0.188, Cr:0.203, W:0.203  0     0     0.5
  3  Zr:0.203, Ti:0.203, V:0.188, Cr:0.203, W:0.203  0     0     0.75
  4  Zr:0.203, Ti:0.203, V:0.188, Cr:0.203, W:0.203  0     0.25  0
  5  Zr:0.203, Ti:0.203, V:0.188, Cr:0.203, W:0.203  0     0.25  0.25
  6  Zr:0.203, Ti:0.203, V:0.188, Cr:0.203, W:0.203  0     0.25  0.5
  7  Zr:0.203, Ti:0.203, V:0.188, Cr:0.203, W:0.203  0     0.25  0.75
  8  Zr:0.203, Ti:0.203, V:0.188, Cr:0.203, W:0.203 

## Create the compositions for the entries that we will use as ECI fitting data

In [ ]:
# Example usage:
element_ranges = {'V': (0.1, 0.9), 'Cr': (0.01, 0.1), 'Ti': (0.01, 0.1), 'Zr': (0.01, 0.05), 'W': (0.01, 0.1)}
num_compositions = 20
compositions = generate_compositions(element_ranges, n = num_compositions,balance_element='V',seed=42)
for i, composition in enumerate(compositions, 1):
    print(f'Composition {i}: {composition}')

## Create random supercells based on each composition, and VASP Relaxation jobs for them 

In [ ]:
db_directory = '/Users/myless/Packages/structure_maker/VCrTiWZr_DBs'
vasp_job_path = '/Users/myless/Packages/structure_maker/CE_Vasp_Jobs_T3'
num_structures = 10
supercell_size = 4
num_atoms = supercell_size**3
structures = []


In [ ]:
k = 0 
if not os.path.exists(vasp_job_path):
    os.makedirs(vasp_job_path)
for i, composition in enumerate(compositions):
    new_atoms, new_atom_dict = closest_composition(composition, num_atoms)
    print(new_atom_dict)
    db_name = '_'.join([f'{key}{value}' for key, value in new_atom_dict.items()]) + '.db'
    generate_supercells = create_random_supercells(new_atom_dict, 3.01, supercell_size, os.path.join(db_directory,db_name), num_structures)
    generated_structures = ase_db_to_pymatgen(os.path.join(db_directory,db_name))
    for j, supercell in enumerate(generated_structures.values()):
        if isinstance(supercell, dict):
            structures.append(supercell)
        else:
            structures.append(supercell.as_dict())
            
        job_path = os.path.join(vasp_job_path, f'job_{k}')
        os.makedirs(job_path, exist_ok=True)
        if isinstance(supercell, dict):
            supercell = Structure.from_dict(supercell)
            supercell = supercell.get_sorted_structure()
        make_vasp_job(supercell.get_sorted_structure(), job_path, kpoints_params=(3,3,3), incar_params=None)
        make_slurm_file(job_path, k, num_gpus=1,omp_threads=10)
        print(f'Job {k} made')
        k += 1 
    


In [ ]:
# write to the db_directory as a json 
with open(os.path.join(db_directory, 'vcrtiwzr_t3_structures.json'), 'w') as f:
    json.dump(structures, f)

# Using CLEASE Probe Structures Function

## Import Bibliotechas

In [1]:
from clease.settings import CEBulk
from clease import NewStructures
from clease.settings import Concentration
from structure_generator import generate_unique_supercell

In [2]:
composition = {'V': 0.8, 'Cr': 0.02, 'Ti': 0.03, 'W': 0.08, 'Zr': 0.07}
supercell_size = (4,4,4)
a = 3.01
structure = generate_unique_supercell(a,supercell_size,composition)

{'V': 0.8, 'Cr': 0.02, 'Ti': 0.03, 'W': 0.08, 'Zr': 0.07}


In [3]:
print(structure)

Full Formula (Zr4 Ti2 V52 Cr1 W5)
Reduced Formula: Zr4Ti2V52CrW5
abc   :   3.010000   3.010000   3.010000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (64)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  V     0.126906  0.669621  0.950347
  1  V     0.035926  0.283739  0.240577
  2  V     0.679665  0.807084  0.156565
  3  V     0.769209  0.183409  0.109478
  4  W     0.40176   0.3105    0.561949
  5  V     0.927955  0.64737   0.427371
  6  V     0.710296  0.46591   0.227384
  7  V     0.512009  0.7574    0.555764
  8  V     0.047662  0.151345  0.106192
  9  V     0.220068  0.90363   0.190929
 10  V     0.431669  0.653858  0.988552
 11  V     0.324447  0.971916  0.472247
 12  V     0.010115  0.375661  0.93039
 13  V     0.36223   0.735355  0.029243
 14  V     0.838437  0.178518  0.365303
 15  V     0.774384  0.692971  0.324594
 16  V     0.089516  0.335629  0.887895
 17  V     0.672512  0.606009  0.96629
 18

In [ ]:
from pymatgen.io.cif import CifWriter
writer = CifWriter(structure)


### CLEASE Graveyard

In [11]:
conc = Concentration(basis_elements=[['V','Cr','Ti','W','Zr']])
conc.set_conc_ranges(ranges=[[(0.8,1.0),(0.01,0.1),(0.01,0.1),(0.01,0.1),(0.01,0.1)]])

In [ ]:
settings = CEBulk(crystalstructure='bcc',
                  a=3.01,
                  size=[4,4,4],
                  concentration=conc,
                  db_name = 'test_vcrtiwzr_db.db',
                  max_cluster_dia=[6.0, 4.5, 4.5])

In [ ]:
ns = NewStructures(settings, generation_number=1, struct_per_gen = 10)
ns.generate_probe_structure()